<a href="https://colab.research.google.com/github/EVA5-Stars/EVA5/blob/master/S6/EVA5_Submission_Session_6_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Objective take best 5th code, and run below versions for 25 epochs and report findings:

>> 1. with L1 + BN
2. with L2 + BN
3. with L1 and L2 with BN
4. with GBN
5. with L1 and L2 with GBN


* *You need to write a single loop or iterator to iterate through these conditions.*

* Draw ONE graph to show the validation accuracy curves for all 5 jobs above. This graph must have proper legends and it should be clear what we are looking at.

* Draw ONE graph to show the loss change curves for all 5 jobs above. This graph must have proper legends and it should be clear what we are looking at.

* Find any 25 misclassified images (combined into single image) for "with GBN" model. You should be using the saved model from the above jobs.  You MUST show the actual and predicted class names.

Submitted By: 
* Gaurav Makkar
* Mohit Bhandari
* Suman Debnath
* Sriram Iyengar




## Import `pytorch` libraries and modules:

In [1]:
from __future__ import print_function
import torch
#import nueral net model in the torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#torchvision is the libraray in torch that has many features
#and we are using datasets and transforms from this library 
from torchvision import datasets, transforms

In [2]:
!rm -rf EVA5
!git clone https://github.com/EVA5-Stars/EVA5.git

Cloning into 'EVA5'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 157 (delta 89), reused 91 (delta 43), pack-reused 0
Receiving objects: 100% (157/157), 620.15 KiB | 888.00 KiB/s, done.
Resolving deltas: 100% (89/89), done.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive/Colab\ Notebooks/EVA5/S6

Mounted at /content/gdrive
EVA5_Submission_Session_6_inprogress.ipynb
EVA5:Submission-Session_6_Model-BatchNormalization.pt
EVA5:Submission-Session_6_Model-GhostBatchNormalization.pt
L1EVA5:Submission-Session_6_Model-BatchNormalization.pt
L1EVA5:Submission-Session_6_Model-GhostBatchNormalization.pt


In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Project Library:

*   Ghost Batch Normalization
*   Misclassified image collection
*   Load the testing and training data with configurabe *batch_size*
*   Plot the Loss and Accuracy Graph
*   Define the structure of the network
*   Setup to create model and summary
*   Load the testing and training data with configurabe *batch_size*
*   Define test and train functions
*   Create a function to *initiate_training* for all conditions





**Ghost Batch Normalization**: 

A technique originally developed for training with very large batch sizes across many accelerators. 

In [6]:
#from EVA5.S6.BatchNorm_GhostBatchNorm import GhostBatchNorm
#from EVA5.S6.BatchNorm_GhostBatchNorm import BatchNorm

**Collecting Misclassified Images**

In [7]:
#from S6.S6_plot_misclassfied_image_finder import misclassified_image_finder              
def misclassified_image_finder(model, model_path, device, train_loader, image_num, msg):
    
    data_iter = iter(test_loader)
    figure = plt.figure()

    plt.title('Misclassified Images: With {}'.format(msg))
   
    for _i in range(image_num):
          data, target = data_iter.next()

          model.load_state_dict(torch.load(model_path)) 
          model.eval()

          data, target = data.to(device), target.to(device)

          output = model(data)
          pred = output.argmax(dim=1, keepdim=True) 
          leng=len(pred)
          for a in range(leng):
              if(pred[a]!=target[a]):                  
                  plt.subplot(5,5,_i+1)
                  plt.axis('off')
                  plt.imshow(data[a].cpu().numpy().squeeze(),cmap='gray_r')
                  

Plot the Loss and Accuracy Graph

In [8]:
from EVA5.S6.S6_plot_stat_loss_acc import plot_stat

Define the structure of the network

In [9]:
from EVA5.S6.S6_Model_structure_create import Net

Setup to create model and summary

In [10]:
from EVA5.S6.S6_Model_structure_create import create_model

Load the testing and training data with configurabe *batch_size*

In [11]:
from EVA5.S6.S6_data_loader import init_train_test_loader

Define test and train functions

In [12]:
from EVA5.S6.S6_train_test_function import train
from EVA5.S6.S6_train_test_function import test   


Create a function to *initiate_training* for all conditions

In [13]:
from torch.optim.lr_scheduler import StepLR
from EVA5.S6.S6_init_training import init_training   


 Initialize train and test loader with Batch Size:128


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Run: Initialization and trainings

Create the model with normal BatchNormalization

In [14]:
model = create_model(device,gbatnor=False)
# Set the Epoch
EPOCHS = 25
model_name_bn = "EVA5:Submission-Session_6_Model-BatchNormalization.pt"
save_path_bn = F"/content/gdrive/My Drive/Colab Notebooks/EVA5/S6/{model_name_bn}"
torch.save(model.state_dict(), save_path_bn)
print("Model Saved: {}".format(model_name_bn))

Create the instance of the Net class with GBN = False

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8            [-1, 8, 12, 12]             128
              ReLU-9            [-1, 8, 12, 12]               0
      BatchNorm2d-10            [-1, 8, 12, 12]              16
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10] 

Create the model with Ghost BatchNormalization

In [15]:
model1 = create_model(device, gbatnor=True)
model_name_gbn = "EVA5:Submission-Session_6_Model-GhostBatchNormalization.pt"
save_path_gbn = F"/content/gdrive/My Drive/Colab Notebooks/EVA5/S6/{model_name_gbn}"
torch.save(model1.state_dict(), save_path_gbn)
print("Model Saved: {}".format(model_name_gbn))

Create the instance of the Net class with GBN = True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
    GhostBatchNorm-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
    GhostBatchNorm-6           [-1, 16, 24, 24]              32
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8            [-1, 8, 12, 12]             128
              ReLU-9            [-1, 8, 12, 12]               0
   GhostBatchNorm-10            [-1, 8, 12, 12]              16
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
   GhostBatchNorm-13           [-1, 16, 10, 10]   

1. Cases


In [ ]:
# Total Loss & Accuracy
total_train_losses = []
total_train_accuracies = []

total_test_losses = []
total_test_accuracies = []
#####
train_loader, test_loader = init_train_test_loader()
# with L1 and BN
Case1=["L1 and BN",model, device, train_loader, EPOCHS, False, 1e-5, False]
# with L2 and BN
Case2=["L2 and BN",model, device, train_loader, EPOCHS,False, None, True]
# with L1, L2 and BN
Case3=["L1, L2 and BN",model, device, train_loader, EPOCHS,  True, 1e-5,True]
# with GBN
Case4=["GBN",model1, device, train_loader, EPOCHS, False, 1e-5,False]
# with L1, L2 and GBN
Case5=["L1, L2 and GBN",model1, device, train_loader, EPOCHS, True, 1e-5, True]

Cases=[Case1,Case2,Case3,Case4,Case5]
for case in Cases:
  # Loss train & test
  train_losses = []
  test_losses = []  
  # Accuracy train & test
  train_acc = []
  test_acc = []
  case_name,model, device, train_loader, epochs, step_lr, l1_lambda, l2_en=case
  print(" Below output is for Case",case_name)
  #print(*case)
  init_training(model, device, train_loader, epochs,train_losses,train_acc,test_losses,test_acc, step_lr=step_lr, l1_lambda=l1_lambda, l2_en=l2_en)
  total_train_losses.extend([train_losses])
  total_train_accuracies.extend([train_acc])
  total_test_losses.extend([test_losses])
  total_test_accuracies.extend([test_acc])
  print(total_test_losses)
  print(total_test_accuracies)

Loss=2.351052761077881 Batch_id=0 Accuracy=8.59:   0%|          | 1/469 [00:00<01:16,  6.13it/s]


 Initialize train and test loader with Batch Size:128
 Below output is for Case L1 and BN


Loss=0.23027123510837555 Batch_id=468 Accuracy=86.80: 100%|██████████| 469/469 [00:55<00:00,  8.48it/s]



 Epoch 0, lr 0.01


Loss=0.27114158868789673 Batch_id=0 Accuracy=96.09:   0%|          | 1/469 [00:00<00:56,  8.28it/s]

...

Loss=0.042164940387010574 Batch_id=468 Accuracy=98.88: 100%|██████████| 469/469 [00:54<00:00,  8.55it/s]



 Epoch 20, lr 0.01


Loss=0.04369354620575905 Batch_id=0 Accuracy=98.44:   0%|          | 1/469 [00:00<00:57,  8.12it/s]


Test set: Average loss: 0.0283, Accuracy: 9922/10000 (99.22%)



Loss=0.03698575496673584 Batch_id=189 Accuracy=99.05:  41%|████      | 190/469 [00:22<00:32,  8.63it/s]

In [ ]:
#Plot for Test Losses for Different Cases
plot_stat(total_test_losses, 'Loss')

In [ ]:
# Plot for Test accuracy for Different Cases
print(total_test_accuracies)
plot_stat(total_test_accuracies, 'Acc')

In [ ]:
misclassified_image_finder(model, save_path_gbn, device, train_loader, EPOCHS, "Ghost Batch Normalization")

## Summary:

A simple DNN with Total params=7,612 was created and trained for 25 epochs with L1, L2 Regularization, Batch Normalization and Ghost Batch Normalization. 

Following are the accuracy observed:

> 1. with L1 + BN => **99.29%**
2. with L2 + BN => **99.31%**
3. with L1 and L2 with BN => **99.47%**
4. with GBN => **98.9%**
5. with L1 and L2 with GBN => **99.33%**

Reference:
* GBN:  https://github.com/apple/ml-cifar-10-faster/blob/master/utils.py

